In [1]:
import os
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import re
import string

c:\Users\Angel\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Specify the directory containing the files
directory = 'D:\\PUCP\\2024-1\\Temas Avanzados en Ciencias de la Computacion\\Proyecto\\tacc-pukyu-yachay\\Corpus-Aprendizaje'

In [3]:
all_files = os.listdir(directory)

In [4]:
es_files = [f for f in all_files if f.endswith('.es')]
quy_files = [f for f in all_files if f.endswith('.quy')]

In [5]:
es_files.sort()
quy_files.sort()

In [6]:
spanish_csv = 'spanish.csv'
quechua_csv = 'quechua.csv'

In [7]:
def read_file_content(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        return file.read().split('\n')

In [8]:
content = read_file_content(os.path.join(directory, es_files[0]))
print(len(content))

34831


In [9]:
spanish_corpus = []
for es_file in es_files:
    filepath = os.path.join(directory, es_file)
    content = read_file_content(filepath)
    spanish_corpus.extend(content)

In [10]:
quechua_corpus = []
for quy_file in quy_files:
    filepath = os.path.join(directory, quy_file)
    content = read_file_content(filepath)
    quechua_corpus.extend(content)

In [11]:
print(len(spanish_corpus))

181572


In [12]:
print(len(quechua_corpus))

181572


In [13]:
def preprocesamiento(lista_cadenas):
    resultado = []
    signos_a_eliminar = string.punctuation + '"' + '¿' + '¡'
    for cadena in lista_cadenas:
        cadena = cadena.translate(str.maketrans('', '', signos_a_eliminar))
        cadena = cadena.lower()
        cadena = cadena.split()
        cadena = ' '.join(cadena)
        resultado.append(cadena)
    return resultado

In [14]:
spanish_corpus = preprocesamiento(spanish_corpus)
quechua_corpus = preprocesamiento(quechua_corpus)

In [15]:
spanish_corpus[0]

'en el principio creó dios los cielos y la tierra'

In [16]:
quechua_corpus[0]

'tukuy imapa qallariyninpi cielokunata hinaspa kay pachata dios unanchaptinmi'

In [17]:
assert len(spanish_corpus) == len(quechua_corpus)

In [18]:
data = [{'es': es, 'quy': quy} for es, quy in zip(spanish_corpus, quechua_corpus)]

In [19]:
resultado = [{'translations': f"{quy} ###>{es}"} for es, quy in zip(spanish_corpus, quechua_corpus)]

In [20]:
resultado[:2]

[{'translations': 'tukuy imapa qallariyninpi cielokunata hinaspa kay pachata dios unanchaptinmi ###>en el principio creó dios los cielos y la tierra'},
 {'translations': 'kay pacha karqa mana formayoq hinaspa mana imayoq ukuuku tutayaq lamar qocham karqa diospa espiritunpas yakukunapa hawanpim muyurqa ###>y la tierra estaba sin orden y vacía y las tinieblas cubrían la superficie del abismo y el espíritu de dios se movía sobre la superficie de las aguas'}]

In [21]:
resultado_df = pd.DataFrame(resultado)
resultado_df.columns = ['translations']

In [22]:
resultado_df.head(2)

,translations
0,tukuy imapa qallariyninpi cielokunata hinaspa ...
1,kay pacha karqa mana formayoq hinaspa mana ima...


In [23]:
train_data, validation_data = train_test_split(resultado, test_size=0.2, random_state=42)

In [24]:
train_dataset = Dataset.from_list(train_data)
validation_dataset = Dataset.from_list(validation_data)

In [25]:
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset
})

In [26]:
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['translations'],
        num_rows: 145257
    })
    validation: Dataset({
        features: ['translations'],
        num_rows: 36315
    })
})


In [27]:
train_df = pd.DataFrame(train_data)
train_df.columns = ['translations']

val_df = pd.DataFrame(validation_data)
val_df.columns = ['translations']

In [28]:
train_df.head()

,translations
0,estados unidos nacionpi wawqenchikmi kay texto...
1,qamqa pasakunankamam wiñaypaqña runata venceru...
2,jarkay ###>obstruir
3,imay urataq ###>a qué hora
4,mana allin ruraqkunam ichaqa kay pachamanta ch...


In [29]:
val_df.head()

,translations
0,chayta ruwaspanchikmi yanapasunchik diosman ‘ ...
1,sumaq yachachiyniyoq ###>número especial
2,kusikunchikmi “diospa samanan punchawman yayku...
3,###>mire la nota
4,chaywanpas hinallam diosmanta willakunaypaq at...


In [30]:
train_df.to_csv('dataset/train.csv', index=False)
val_df.to_csv('dataset/validation.csv', index=False)

In [31]:
train_dataset_final = Dataset.from_csv('dataset/train.csv')

Generating train split: 145257 examples [00:00, 285045.12 examples/s]


In [32]:
train_dataset_final['translations'][:5]

['estados unidos nacionpi wawqenchikmi kay textopa nisqanta kasukurqa ###>un hermano de estados unidos tomó a pecho este consejo',
 'qamqa pasakunankamam wiñaypaqña runata vencerunki rikchaynintapas tikraruspaykim paytaqa qarqorunki ###>prevaleces para siempre contra él y se va cambias su apariencia y lo despides',
 'jarkay ###>obstruir',
 'imay urataq ###>a qué hora',
 'mana allin ruraqkunam ichaqa kay pachamanta chinkachisqa kanqaku traicionaqkunam ichaqa qora pilarusqa hina kay pachamanta qechusqa kanqaku ###>pero los impíos serán cortados de la tierra y los pérfidos serán desarraigados de ella']